In [2]:
1+1

2

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import time
import random
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [10]:

# Standardizes formatting for prices
def clean_price(price_text):
    if not price_text: return "N/A"
    return "".join(filter(str.isdigit, price_text))

URL = "https://www.airbnb.com/s/Paris/homes?refinement_paths%5B%5D=%2Fhomes&date_picker_type=calendar&search_type=search_query&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2026-02-01&monthly_length=3&monthly_end_date=2026-05-01&price_filter_input_type=2&channel=EXPLORE&price_filter_num_nights=5&min_bedrooms=3&selected_filter_order%5B%5D=min_bedrooms%3A3&update_selected_filters=false"

chrome_options = Options()
# chrome_options.add_argument("--headless")
service = ChromeService(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

all_data = []
target_count = 50

try:
    driver.get(URL)
    wait = WebDriverWait(driver, 15)

    while len(all_data) < target_count:
        print(f"--- Scraping Page (Collected: {len(all_data)}) ---")
        
        # 1. Wait for cards
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[data-testid='card-container']")))
        
        # 2. Collect URLs from current page
        listings = driver.find_elements(By.CSS_SELECTOR, "div[data-testid='card-container']")
        listing_urls = []
        
        for l in listings:
            try:
                url = l.find_element(By.TAG_NAME, "a").get_attribute("href")
                listing_urls.append(url)
            except:
                continue

        # 3. Store main window handle
        main_window = driver.current_window_handle

        # 4. Visit each URL
        for url in listing_urls:
            if len(all_data) >= target_count:
                break
            
            try:
                # Open new tab via JS (Faster/Cleaner)
                driver.execute_script(f"window.open('{url}', '_blank');")
                driver.switch_to.window(driver.window_handles[-1])

                # --- STABLE SCRAPING LOGIC ---
                
                # Wait for the H1 (Title) to ensure page load
                wait.until(EC.presence_of_element_located((By.TAG_NAME, "h1")))
                
                # A. Scrape Name
                try:
                    name = driver.find_element(By.TAG_NAME, "h1").text
                except:
                    name = "No Title"

                # B. Scrape Superhost (Look for the text, not the class)
                # This XPath searches the whole page for the text "Superhost"

                superhost_indicators = driver.find_elements(By.XPATH, "//*[contains(text(), 'Superhost')]")
                is_superhost = "Yes" if len(superhost_indicators) > 0 else "No"

                # C. Scrape Price (This is tricky on Detail pages, easier on Search pages)
                # We look for the "price per night" element usually found in the booking sidebar
                price = "N/A"
                try:
                    # Look for element with "$" text inside the booking sidebar or sticky header
                    price_element = driver.find_element(By.XPATH, "//div[contains(@data-section-id, 'BOOK_IT')]//span[contains(text(), '$')]")
                    price = price_element.text
                except:
                    # Fallback: Capture it from the source loosely
                    pass

                all_data.append({
                    "Name": name,
                    "Price": price,
                    "Superhost": is_superhost,
                    "URL": url
                })
                
                print(f"[{len(all_data)}] {is_superhost} | {name[:30]}...")

                # Random sleep to look human
                time.sleep(random.uniform(1.5, 3.0))

                # Close tab and return
                driver.close()
                driver.switch_to.window(main_window)
                
            except Exception as e:
                print(f"Skipping listing due to error: {e}")
                if len(driver.window_handles) > 1:
                    driver.close()
                    driver.switch_to.window(main_window)
                continue

        # 5. Pagination Logic
        if len(all_data) < target_count:
            try:
                next_btn = driver.find_element(By.CSS_SELECTOR, "a[aria-label='Next']")
                driver.execute_script("arguments[0].click();", next_btn)
                time.sleep(5) # Allow next page to load
            except:
                print("No next page found or target reached.")
                break

finally:
    print(f"\nCompleted! Total scraped: {len(all_data)}")
    driver.quit()

--- Scraping Page (Collected: 0) ---
[1] Yes | Large terrace, air conditionin...
--- Scraping Page (Collected: 1) ---
[2] Yes | Sweett | Monsigny II...
[3] Yes | Superb AC apartment Center of ...
[4] Yes | Babylon Grands Boulevards...
[5] Yes | Luxurious 3BD with 3BR and AC...
[6] Yes | Merveil - Luxury Suite - Marai...
[7] Yes | Merveil - Signature Suite - Ei...
[8] Yes | LivinParis NEW Luxury A/C 3 Be...
[9] Yes | Edgar Suites Martyrs - Lebas 1...
[10] Yes | Joie Paris Penthouse Arago...
[11] Yes | Edgar Suites Louvre - Day 101...
[12] Yes | Edgar Suites Louvre - Day 303...
[13] Yes | Edgar Suites Tour Eiffel – Car...
[14] Yes | The Residence old Marais: Pati...

Completed! Total scraped: 14


KeyboardInterrupt: 